In [ ]:
import cv2
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import random

# Deep Learning
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense,  SeparableConv2D, Conv2D , Conv3D, MaxPool3D, MaxPool2D , Flatten, Input, Concatenate, concatenate , Dropout , BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from bibliotheque_projet import import_data, separation_data, creation_model, fit_model, creation_y

In [ ]:
labels = ["1" , "2" , "3" , "4"]
electrodes = ["C3" , "C4" , "Cz"]

racine_scalo = "/content/drive/MyDrive/Projet_E4_Interface_Cerveau_Machine/Bases de données EEG/Bases de données/bdd_scalo"
racine_spectro = "/content/drive/MyDrive/Projet_E4_Interface_Cerveau_Machine/Bases de données EEG/Bases de données/bdd_spectro"

# Importation des données

In [ ]:
data_scalo = import_data(electrodes[0], racine_scalo, labels)
X_train1, X_test1, X_val1, y_train1, y_test1, y_val1 = separation_data(data_scalo)
print(X_train1.shape)

data_spectro = import_data(electrodes[0], racine_spectro, labels)
X_train2, X_test2, X_val2, y_train2, y_test2, y_val2 = separation_data(data_spectro)
print(X_train2.shape)

data_scalo2 = import_data(electrodes[1], racine_scalo, labels)
X_train3, X_test3, X_val3, y_train3, y_test3, y_val3 = separation_data(data_scalo)
print(X_train3.shape)

data_spectro2 = import_data(electrodes[1], racine_spectro, labels)
X_train4, X_test4, X_val4, y_train4, y_test4, y_val4 = separation_data(data_spectro)
print(X_train3.shape)

data_scalo3 = import_data(electrodes[2], racine_scalo, labels)
X_train5, X_test5, X_val5, y_train5, y_test5, y_val5 = separation_data(data_scalo)
print(X_train5.shape)

data_spectro3 = import_data(electrodes[2], racine_spectro, labels)
X_train6, X_test6, X_val6, y_train6, y_test6, y_val6 = separation_data(data_spectro)
print(X_train6.shape)

(326, 250, 250, 3)
(326, 250, 250, 3)
(326, 250, 250, 3)
(326, 250, 250, 3)
(326, 250, 250, 3)
(326, 250, 250, 3)


In [ ]:
y_train = creation_y(y_train1)
y_test = creation_y(y_test1)
y_val = creation_y(y_val1)

# On définit notre premier modèle

Notre modèle va prendre 6 entrées:
- l'image du scalogramme et du spectrogramme de l'électrode C3,
- l'image du scalogramme et du spectrogramme de l'électrode C4,
- l'image du scalogramme et du spectrogramme de l'électrode Cz

In [ ]:
shape = np.shape(X_train1)

input_1 = Input(shape=shape[1:])
input_2 = Input(shape=shape[1:])
input_3 = Input(shape=shape[1:])
input_4 = Input(shape=shape[1:])
input_5 = Input(shape=shape[1:])
input_6 = Input(shape=shape[1:])

model = concatenate([input_1, input_2, input_3, input_4, input_5, input_6], axis = -1)

model = Conv2D(16, (3,3)) (model)
model = Activation("relu") (model)
model = MaxPool2D(16, (2,2)) (model)
model = Activation('relu') (model)
model = Dense(16) (model)
model = Dropout(0.2) (model)

model = Conv2D(32, (3,3)) (model)
model = Activation("relu") (model)
model = MaxPool2D(32, (2,2)) (model)
model = Activation('relu') (model)
model = Dense(32) (model)
model = Dropout(0.2) (model)

model = Conv2D(64, (3,3)) (model)
model = Activation("relu") (model)
model = MaxPool2D(64, (2,2)) (model)
model = Activation('relu') (model)
model = Dense(64) (model)
model = Dropout(0.25) (model)

model = Flatten() (model)
model = Dense(64) (model)
model = Activation('relu') (model)
model = Dropout(0.5) (model)
model = Dense(4) (model)

output = Activation('softmax') (model)

model = Model([input_1, input_3, input_5], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 250, 250, 9)  0           input_13[0][0]                   
                                                                 input_15[0][0]               

# Entrainement d'un modele qui prend en entrée la prédiction de 3 autres

**Entraînement des 3 modèles**

In [ ]:
shape = np.shape(X_train1)

input_1 = Input(shape=shape[1:])
input_2 = Input(shape=shape[1:])
input_3 = Input(shape=shape[1:])
input_4 = Input(shape=shape[1:])
input_5 = Input(shape=shape[1:])
input_6 = Input(shape=shape[1:])

#modele 1
model_1 = concatenate([input_1, input_2], axis = -1)
model_1 = Conv2D(32, (3,3)) (model_1)
model_1 = Activation("relu") (model_1)
model_1 = MaxPool2D(32, (2,2)) (model_1)
model_1 = Activation('relu') (model_1)
model_1 = Dense(32) (model_1)
model_1 = Dropout(0.2) (model_1)

model_1 = Conv2D(64, (3,3)) (model_1)
model_1 = Activation("relu") (model_1)
model_1 = MaxPool2D(64, (2,2)) (model_1)
model_1 = Activation('relu') (model_1)
model_1 = Dense(64) (model_1)
model_1 = Dropout(0.25) (model_1)

model_1 = Flatten() (model_1)
model_1 = Dense(64) (model_1)
model_1 = Activation('relu') (model_1)
model_1 = Dropout(0.5) (model_1)
model_1 = Dense(4) (model_1)

output_1 = Activation('softmax') (model_1)

model_1 = Model([input_1, input_2], output_1)

#modele2
model_2 = concatenate([input_3, input_4], axis = -1)
model_2 = Conv2D(32, (3,3)) (model_2)
model_2 = Activation("relu") (model_2)
model_2 = MaxPool2D(32, (2,2)) (model_2)
model_2 = Activation('relu') (model_2)
model_2 = Dense(32) (model_2)
model_2 = Dropout(0.2) (model_2)

model_2 = Conv2D(64, (3,3)) (model_2)
model_2 = Activation("relu") (model_2)
model_2 = MaxPool2D(64, (2,2)) (model_2)
model_2 = Activation('relu') (model_2)
model_2 = Dense(64) (model_2)
model_2 = Dropout(0.25) (model_2)

model_2 = Flatten() (model_2)
model_2 = Dense(64) (model_2)
model_2 = Activation('relu') (model_2)
model_2 = Dropout(0.5) (model_2)
model_2 = Dense(4) (model_2)

output_2 = Activation('softmax') (model_2)

model_2 = Model([input_3, input_4], output_2)

#modele3
model_3 = concatenate([input_5, input_6], axis = -1)
model_3 = Conv2D(32, (3,3)) (model_3)
model_3 = Activation("relu") (model_3)
model_3 = MaxPool2D(32, (2,2)) (model_3)
model_3 = Activation('relu') (model_3)
model_3 = Dense(32) (model_3)
model_3 = Dropout(0.2) (model_3)

model_3 = Conv2D(64, (3,3)) (model_3)
model_3 = Activation("relu") (model_3)
model_3 = MaxPool2D(64, (2,2)) (model_3)
model_3 = Activation('relu') (model_3)
model_3 = Dense(64) (model_3)
model_3 = Dropout(0.25) (model_3)

model_3 = Flatten() (model_3)
model_3 = Dense(64) (model_3)
model_3 = Activation('relu') (model_3)
model_3 = Dropout(0.5) (model_3)
model_3 = Dense(4) (model_3)

output_3 = Activation('softmax') (model_3)

model_3 = Model([input_5, input_6], output_3)

In [ ]:
epochs = 10
batch_size = int(len(X_train1)/ 12)

datagen = ImageDataGenerator( shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train1)

model_1.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model_1.fit(datagen.flow([X_train1, X_train2], y_train, batch_size = 32), 
          steps_per_epoch= int(len(X_train1) / 32),
          epochs=epochs,
          validation_data = ([X_val1, X_val2], y_val))

Epoch 1/10
10/10 [==============================] - 38s 487ms/step - loss: 6.7069 - accuracy: 0.2778 - val_loss: 2.5282 - val_accuracy: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 4s 435ms/step - loss: 1.4524 - accuracy: 0.2577 - val_loss: 1.4211 - val_accuracy: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 4s 431ms/step - loss: 1.4011 - accuracy: 0.2730 - val_loss: 2.7067 - val_accuracy: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 4s 427ms/step - loss: 1.3953 - accuracy: 0.2613 - val_loss: 2.6554 - val_accuracy: 0.0000e+00
Epoch 5/10
10/10 [==============================] - 4s 425ms/step - loss: 1.3863 - accuracy: 0.2628 - val_loss: 1.8516 - val_accuracy: 0.0000e+00
Epoch 6/10
10/10 [==============================] - 4s 424ms/step - loss: 1.3863 - accuracy: 0.2665 - val_loss: 1.7018 - val_accuracy: 0.0000e+00
Epoch 7/10
10/10 [==============================] - 4s 477ms/step - loss: 1.3865 - accuracy: 0.2418 - val_loss: 1.6551 - va

In [ ]:
model_1.save('model_C3.h5')

In [ ]:
epochs = 10
batch_size = int(len(X_train1)/ 12)

datagen = ImageDataGenerator( shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train3)

model_2.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model_2.fit(datagen.flow([X_train3, X_train4], y_train, batch_size = 32), 
          steps_per_epoch= int(len(X_train3) / 32),
          epochs=epochs,
          validation_data = ([X_val3, X_val4], y_val))

Epoch 1/10
10/10 [==============================] - 5s 450ms/step - loss: 5.5476 - accuracy: 0.2327 - val_loss: 8.5635 - val_accuracy: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 4s 426ms/step - loss: 1.9554 - accuracy: 0.1969 - val_loss: 1.3863 - val_accuracy: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 4s 422ms/step - loss: 1.3902 - accuracy: 0.3003 - val_loss: 1.3531 - val_accuracy: 0.6000
Epoch 4/10
10/10 [==============================] - 4s 428ms/step - loss: 1.3862 - accuracy: 0.2862 - val_loss: 1.3863 - val_accuracy: 0.2000
Epoch 5/10
10/10 [==============================] - 4s 417ms/step - loss: 1.3862 - accuracy: 0.2736 - val_loss: 1.3879 - val_accuracy: 0.0000e+00
Epoch 6/10
10/10 [==============================] - 5s 458ms/step - loss: 1.3863 - accuracy: 0.2560 - val_loss: 1.3877 - val_accuracy: 0.0000e+00
Epoch 7/10
10/10 [==============================] - 4s 466ms/step - loss: 1.3862 - accuracy: 0.2783 - val_loss: 1.3882 - val_accurac

In [ ]:
model_2.save('model_C4.h5')

In [ ]:
epochs = 10
batch_size = int(len(X_train1)/ 12)

datagen = ImageDataGenerator( shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train5)

model_3.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model_3.fit(datagen.flow([X_train5, X_train6], y_train, batch_size = 32), 
          steps_per_epoch= int(len(X_train5) / 32),
          epochs=epochs,
          validation_data = ([X_val5, X_val6], y_val))

Epoch 1/10
10/10 [==============================] - 5s 498ms/step - loss: 5.3041 - accuracy: 0.1990 - val_loss: 1.6418 - val_accuracy: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 4s 423ms/step - loss: 1.5206 - accuracy: 0.2431 - val_loss: 1.3489 - val_accuracy: 0.2000
Epoch 3/10
10/10 [==============================] - 4s 422ms/step - loss: 1.4147 - accuracy: 0.2680 - val_loss: 1.3845 - val_accuracy: 0.8000
Epoch 4/10
10/10 [==============================] - 4s 426ms/step - loss: 1.3863 - accuracy: 0.2499 - val_loss: 1.3836 - val_accuracy: 0.8000
Epoch 5/10
10/10 [==============================] - 4s 423ms/step - loss: 1.3903 - accuracy: 0.2434 - val_loss: 1.3831 - val_accuracy: 0.8000
Epoch 6/10
10/10 [==============================] - 4s 425ms/step - loss: 1.3864 - accuracy: 0.2374 - val_loss: 1.3840 - val_accuracy: 0.8000
Epoch 7/10
10/10 [==============================] - 4s 414ms/step - loss: 1.3862 - accuracy: 0.2522 - val_loss: 1.3841 - val_accuracy: 0.8000
Ep

In [ ]:
model_3.save('model_Cz.h5')

In [ ]:
y_pred1 = model_1.predict([X_train1, X_train2])
y_pred2 = model_2.predict([X_train3, X_train4])
y_pred3 = model_3.predict([X_train5, X_train6])

In [ ]:
y_pred1.shape

(326, 4)

In [ ]:
shape_finale = y_pred1.shape

input_7 = Input(shape_finale[1])
input_8 = Input(shape_finale[1])
input_9 = Input(shape_finale[1])

model = concatenate([input_7, input_8, input_9], axis = -1)

model = Dense(32) (model)
model = Activation('relu') (model)
model = Dense(64) (model)
model = Activation('relu') (model)
model = Dense(128) (model)
model = Activation('relu') (model)
model = Dense(4) (model)
output = Activation('softmax') (model)

model = Model([input_7, input_8, input_9], output)
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_52 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
input_53 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
input_54 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
concatenate_29 (Concatenate)    (None, 12)           0           input_52[0][0]                   
                                                                 input_53[0][0]            

In [ ]:
epochs = 10
batch_size = int(len(X_train1)/ 12)

datagen = ImageDataGenerator( shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train3)

model.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit([y_pred1, y_pred2, y_pred3], y_train, batch_size = 32, 
          steps_per_epoch= int(len(y_pred1) / 32),
          epochs=epochs)

Epoch 1/10
10/10 [==============================] - 0s 3ms/step - loss: 1.3880 - accuracy: 0.2312
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 1.3852 - accuracy: 0.2687
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 1.3873 - accuracy: 0.2283
Epoch 4/10
10/10 [==============================] - 0s 2ms/step - loss: 1.3849 - accuracy: 0.2932
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 1.3873 - accuracy: 0.2562
Epoch 6/10
10/10 [==============================] - 0s 2ms/step - loss: 1.3855 - accuracy: 0.2831
Epoch 7/10
10/10 [==============================] - 0s 2ms/step - loss: 1.3842 - accuracy: 0.2793
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 1.3864 - accuracy: 0.2832
Epoch 9/10
10/10 [==============================] - 0s 3ms/step - loss: 1.3796 - accuracy: 0.2883
Epoch 10/10
10/10 [==============================] - 0s 3ms/step - loss: 1.3848 - accuracy: 0.2543


In [ ]:
y_fin1 = model_1.predict([X_test1, X_test2])
y_fin2 = model_2.predict([X_test3, X_test4])
y_fin3 = model_3.predict([X_test5, X_test6])

y_pred_final = model.predict([y_fin1, y_fin2, y_fin3])

In [ ]:
np.argmax(y_pred_final, axis = 1)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])